In [2]:
import warnings
warnings.filterwarnings("ignore")

# Importing Basic libraries

In [3]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

In [4]:
df = pd.read_csv('./YoutubeCommentsDataSet.csv')
df.head()

,Comment,Sentiment
0,lets not forget that apple pay in 2014 require...,neutral
1,here in nz 50 of retailers don’t even have con...,negative
2,i will forever acknowledge this channel with t...,positive
3,whenever i go to a place that doesn’t take app...,negative
4,apple pay is so convenient secure and easy to ...,positive


In [5]:
df.isnull().sum()

Comment      44
Sentiment     0
dtype: int64

In [6]:
df.dropna(inplace=True)

In [7]:
print(f"Percent of duplication in the dataset: {round(df.duplicated().sum() / len(df), 3) * 100}%")

Percent of duplication in the dataset: 2.7%


Since the duplication is less than 3% of the original dataset, we can just drop it

In [8]:
df.drop_duplicates(inplace=True)

In [9]:
df.head()

,Comment,Sentiment
0,lets not forget that apple pay in 2014 require...,neutral
1,here in nz 50 of retailers don’t even have con...,negative
2,i will forever acknowledge this channel with t...,positive
3,whenever i go to a place that doesn’t take app...,negative
4,apple pay is so convenient secure and easy to ...,positive


In [10]:
df['Sentiment'].value_counts()

Sentiment
positive    11054
neutral      4503
negative     2317
Name: count, dtype: int64

In [11]:
df['Sentiment'].replace({'positive': 1, 
                         'negative': -1,
                         'neutral': 0}, inplace=True)

df.head()

,Comment,Sentiment
0,lets not forget that apple pay in 2014 require...,0
1,here in nz 50 of retailers don’t even have con...,-1
2,i will forever acknowledge this channel with t...,1
3,whenever i go to a place that doesn’t take app...,-1
4,apple pay is so convenient secure and easy to ...,1


## Data Cleaning

1. LowerCase all the text

In [12]:
df['Comment'] = df['Comment'].str.lower()

2. Tokenization 

In [13]:
from nltk.tokenize import word_tokenize
df['Comment'] = df['Comment'].apply(word_tokenize)

3. Removing Punctuation

In [14]:
import re

df['Comment'] = df['Comment'].apply(lambda text: [re.sub(r'\W', ' ', i) for i in text]) 
# keeps words, numbers and spaces, removes punctuations

4. Removing basic english words

In [15]:
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))  
df['Comment'] = df['Comment'].apply(lambda tokens: [word for word in tokens if word not in stop_words])

# MODEL TIME!!!

### Transforming the comments feature

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer

vector = TfidfVectorizer(max_features=10000)

df['Comment'] = df['Comment'].apply(lambda tokens: ' '.join(tokens)) # convert list of tokens to string
X = vector.fit_transform(df['Comment'])

X = pd.DataFrame(X.toarray(), columns=vector.get_feature_names_out())

print("TF-IDF Features:")
print(X.head())


TF-IDF Features:
   000  0000  000000  0001  001  0018   01  010   02  0212  ...  오늘은  음식이  \
0  0.0   0.0     0.0   0.0  0.0   0.0  0.0  0.0  0.0   0.0  ...  0.0  0.0   
1  0.0   0.0     0.0   0.0  0.0   0.0  0.0  0.0  0.0   0.0  ...  0.0  0.0   
2  0.0   0.0     0.0   0.0  0.0   0.0  0.0  0.0  0.0   0.0  ...  0.0  0.0   
3  0.0   0.0     0.0   0.0  0.0   0.0  0.0  0.0  0.0   0.0  ...  0.0  0.0   
4  0.0   0.0     0.0   0.0  0.0   0.0  0.0  0.0  0.0   0.0  ...  0.0  0.0   

    정말   진짜  편의점  𝗧𝗿𝗲𝗻𝗱𝗶𝗻𝗴  𝗮𝗱𝘂𝗹𝘁  𝗼𝗻𝗹𝘆  𝗽𝗼𝗿𝗻  𝘃𝗶𝗱𝗲𝗼𝘀  
0  0.0  0.0  0.0       0.0    0.0   0.0   0.0     0.0  
1  0.0  0.0  0.0       0.0    0.0   0.0   0.0     0.0  
2  0.0  0.0  0.0       0.0    0.0   0.0   0.0     0.0  
3  0.0  0.0  0.0       0.0    0.0   0.0   0.0     0.0  
4  0.0  0.0  0.0       0.0    0.0   0.0   0.0     0.0  

[5 rows x 10000 columns]


In [17]:
from sklearn.model_selection import train_test_split as tts 

x_train, x_test, y_train, y_test = tts(X, df['Sentiment'], test_size=0.2, random_state=42)

In [18]:
from sklearn.ensemble import RandomForestClassifier

classWeights = {-1: 5, 0: 3, 1: 1}
model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight=classWeights)

model.fit(x_train, y_train)

RandomForestClassifier(class_weight={-1: 5, 0: 3, 1: 1}, random_state=42)

In [19]:
pred = model.predict(x_test)

In [20]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(f"Accuracy: {accuracy_score(y_test, pred)}")
print("\nConfusion Matrix:\n", confusion_matrix(y_test, pred))
print("\nClassification Report:\n", classification_report(y_test, pred))

Accuracy: 0.735944055944056

Confusion Matrix:
 [[ 108  118  230]
 [  42  526  338]
 [  20  196 1997]]

Classification Report:
               precision    recall  f1-score   support

          -1       0.64      0.24      0.35       456
           0       0.63      0.58      0.60       906
           1       0.78      0.90      0.84      2213

    accuracy                           0.74      3575
   macro avg       0.68      0.57      0.59      3575
weighted avg       0.72      0.74      0.71      3575



Accuracy

1. RF w/o class weights- 73.11%
2. RF w class weights - 73.56%